In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config('spark.shuffle.useOldFetchProtocol', 'true'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

#### Load loans data

In [2]:
loans_raw_df = spark.read \
.format("csv") \
.option("header", True) \
.option("inferSchema", True) \
.load("/user/itv018355/lendingclubproject/raw/loans_data_csv")

In [3]:
loans_raw_df

+--------+--------------------+---------+-----------+---------+--------+-----------+--------+------------------+------------------+--------------------+
| loan_id|           member_id|loan_amnt|funded_amnt|     term|int_rate|installment| issue_d|       loan_status|           purpose|               title|
+--------+--------------------+---------+-----------+---------+--------+-----------+--------+------------------+------------------+--------------------+
|69642551|2209556c19f75a204...|  10000.0|    10000.0|36 months|   10.78|     326.35|Jan-2016|       Charged Off|       credit_card|Credit card refin...|
|69336966|9d0d53b8de898b446...|  12000.0|    12000.0|36 months|   11.99|     398.52|Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|
|69226186|1a978870f4a9b4b43...|  28000.0|    28000.0|36 months|    7.49|     870.85|Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|
|69084024|c66ccc99ff4372809...|  16500.0|    16500.0|36 months|     9.8|     530.87|Jan-2016|        Fully Paid|debt_consolidation|                null|
|69422635|91d00a422248120a4...|  21000.0|    21000.0|60 months|   10.78|      454.3|Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|
|69542436|b7539a5579e723b21...|  21000.0|    21000.0|60 months|   14.85|     497.94|Feb-2016|       Charged Off|debt_consolidation|  Debt consolidation|
|69492501|2fca945a60ef9246c...|  29000.0|    29000.0|36 months|   11.48|     956.03|Jan-2016|           Current|       credit_card|Credit card refin...|
|69522527|8a6dababd11fd592f...|  33000.0|    33000.0|36 months|   13.44|    1118.91|Jan-2016|        Fully Paid|       credit_card|Credit card refin...|
|68781653|3af72ce29e878edd7...|  18000.0|    18000.0|60 months|   13.44|     413.63|Jan-2016|        Fully Paid|       credit_card|Credit card refin...|
|69371334|4395baca2ea2d58a8...|  16000.0|    16000.0|60 months|    7.91|     323.74|Jan-2016|        Fully Paid|  home_improvement|    Home improvement|
|68714098|45ac50a4b31e7c191...|  15000.0|    15000.0|36 months|    6.99|     463.09|Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|
|69622505|a6a1b2d34c1f8d409...|  35000.0|    35000.0|60 months|   19.89|     925.15|Jan-2016|Late (31-120 days)|debt_consolidation|  Debt consolidation|
|69084020|aa1795bf3b4e526f4...|  12000.0|    12000.0|36 months|   12.88|     403.64|Jan-2016|       Charged Off|debt_consolidation|                null|
|69552651|8e0e745628d1fb06e...|  24000.0|    24000.0|60 months|   13.99|     558.32|Jan-2016|           Current|debt_consolidation|  Debt consolidation|
|69084037|232386e4de5f4542b...|   7200.0|     7200.0|36 months|   11.48|     237.36|Jan-2016|        Fully Paid|    major_purchase|                null|
|69612464|2d3472c91f8b2a006...|   2500.0|     2500.0|36 months|    8.49|      78.91|Jan-2016|       Charged Off|           medical|    Medical expenses|
|69562380|9abeb83f93fd934ff...|  22250.0|    22250.0|60 months|   19.89|     588.13|Jan-2016|           Current|       credit_card|Credit card refin...|
|69422542|cb18b4a8c027d239c...|   7200.0|     7200.0|36 months|     9.8|     231.65|Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|
|69652408|a91eb99321edd2864...|   1500.0|     1500.0|36 months|   13.99|      51.26|Jan-2016|        Fully Paid|               car|       Car financing|
|68756432|ee2a6a369b2fe6170...|  30000.0|    30000.0|60 months|     9.8|     634.47|Jan-2016|           Current|       credit_card|Credit card refin...|
+--------+--------------------+---------+-----------+---------+--------+-----------+--------+------------------+------------------+--------------------+
only showing top 20 rows

#### in one go, we are setting the right column names along with the right data types

In [4]:
loans_schema = 'loan_id string, member_id string, loan_amount float, funded_amount float, loan_term_months string, interest_rate float, monthly_installment float, issue_date string, loan_status string, loan_purpose string, loan_title string'

In [5]:
loans_raw_df = spark.read \
.format("csv") \
.option("header",True) \
.schema(loans_schema) \
.load("/user/itv018355/lendingclubproject/raw/loans_data_csv")

In [6]:
loans_raw_df.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amount: float (nullable = true)
 |-- funded_amount: float (nullable = true)
 |-- loan_term_months: string (nullable = true)
 |-- interest_rate: float (nullable = true)
 |-- monthly_installment: float (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- loan_purpose: string (nullable = true)
 |-- loan_title: string (nullable = true)



#### let's insert a ingest date which has the current timestamp

In [7]:
from pyspark.sql.functions import current_timestamp

In [8]:
loans_df_ingestd = loans_raw_df.withColumn("ingest_date", current_timestamp())

In [9]:
loans_df_ingestd

+--------+--------------------+-----------+-------------+----------------+-------------+-------------------+----------+------------------+------------------+--------------------+--------------------+
| loan_id|           member_id|loan_amount|funded_amount|loan_term_months|interest_rate|monthly_installment|issue_date|       loan_status|      loan_purpose|          loan_title|         ingest_date|
+--------+--------------------+-----------+-------------+----------------+-------------+-------------------+----------+------------------+------------------+--------------------+--------------------+
|69642551|2209556c19f75a204...|    10000.0|      10000.0|       36 months|        10.78|             326.35|  Jan-2016|       Charged Off|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|69336966|9d0d53b8de898b446...|    12000.0|      12000.0|       36 months|        11.99|             398.52|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69226186|1a978870f4a9b4b43...|    28000.0|      28000.0|       36 months|         7.49|             870.85|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69084024|c66ccc99ff4372809...|    16500.0|      16500.0|       36 months|          9.8|             530.87|  Jan-2016|        Fully Paid|debt_consolidation|                null|2025-06-17 03:40:...|
|69422635|91d00a422248120a4...|    21000.0|      21000.0|       60 months|        10.78|              454.3|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69542436|b7539a5579e723b21...|    21000.0|      21000.0|       60 months|        14.85|             497.94|  Feb-2016|       Charged Off|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69492501|2fca945a60ef9246c...|    29000.0|      29000.0|       36 months|        11.48|             956.03|  Jan-2016|           Current|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|69522527|8a6dababd11fd592f...|    33000.0|      33000.0|       36 months|        13.44|            1118.91|  Jan-2016|        Fully Paid|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|68781653|3af72ce29e878edd7...|    18000.0|      18000.0|       60 months|        13.44|             413.63|  Jan-2016|        Fully Paid|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|69371334|4395baca2ea2d58a8...|    16000.0|      16000.0|       60 months|         7.91|             323.74|  Jan-2016|        Fully Paid|  home_improvement|    Home improvement|2025-06-17 03:40:...|
|68714098|45ac50a4b31e7c191...|    15000.0|      15000.0|       36 months|         6.99|             463.09|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69622505|a6a1b2d34c1f8d409...|    35000.0|      35000.0|       60 months|        19.89|             925.15|  Jan-2016|Late (31-120 days)|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69084020|aa1795bf3b4e526f4...|    12000.0|      12000.0|       36 months|        12.88|             403.64|  Jan-2016|       Charged Off|debt_consolidation|                null|2025-06-17 03:40:...|
|69552651|8e0e745628d1fb06e...|    24000.0|      24000.0|       60 months|        13.99|             558.32|  Jan-2016|           Current|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69084037|232386e4de5f4542b...|     7200.0|       7200.0|       36 months|        11.48|             237.36|  Jan-2016|        Fully Paid|    major_purchase|                null|2025-06-17 03:40:...|
|69612464|2d3472c91f8b2a006...|     2500.0|       2500.0|       36 months|         8.49|              78.91|  Jan-2016|       Charged Off|           medical|    Medical expenses|2025-06-17 03:40:...|
|69562380|9abeb83f93fd934ff...|    22250.0|      22250.0|       60 months|        19.89|             588.13|  Jan-2016|           Current|       credit_card|Credit card refin...|2025-06-17 03:40:...|


#### let us create a temporary view


In [10]:
loans_df_ingestd.createOrReplaceTempView("loans")

In [11]:
spark.sql("select count(*) from loans")

+--------+
|count(1)|
+--------+
| 2260701|
+--------+

#### let's say we have to remove any nulls from loan_amount till loan_title


#### how many rows have loan_amount as null


In [12]:
spark.sql("select * from loans where loan_amount IS NULL")

+--------------------+--------------------+-----------+-------------+----------------+-------------+-------------------+----------+-----------+------------+----------+--------------------+
|             loan_id|           member_id|loan_amount|funded_amount|loan_term_months|interest_rate|monthly_installment|issue_date|loan_status|loan_purpose|loan_title|         ingest_date|
+--------------------+--------------------+-----------+-------------+----------------+-------------+-------------------+----------+-----------+------------+----------+--------------------+
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c44298fc1c149...|       null|         null|            null|         null|               null|      null|       null|        null|      null|2025-06-17 03:40:...|
|Total amount fund...|e3b0c4429

In [13]:
spark.sql("select count(*) from loans where loan_amount IS NULL")

+--------+
|count(1)|
+--------+
|      33|
+--------+

#### if any of these is null, drop the entire row


In [14]:
columns_to_check = ["loan_amount","funded_amount","loan_term_months","interest_rate","monthly_installment","issue_date","loan_status","loan_purpose"]

In [15]:
loans_filtered_df = loans_df_ingestd.na.drop(subset= columns_to_check)

In [16]:
loans_filtered_df.count()

2260667

#### Create a table

In [17]:
loans_filtered_df.createOrReplaceTempView("loans")

#### next thing i want to do- instead of loan_term_months i want to have loan_term_years
#### i want it integer rather than a string
#### use regex

In [18]:
from pyspark.sql.functions import regexp_replace, col

In [19]:
loans_term_modified_df = loans_filtered_df.withColumn("loan_term_months", (regexp_replace(col("loan_term_months")," months","").cast("int") / 12).cast("int")).withColumnRenamed("loan_term_months","loan_term_years")

In [20]:
loans_term_modified_df

+--------+--------------------+-----------+-------------+---------------+-------------+-------------------+----------+------------------+------------------+--------------------+--------------------+
| loan_id|           member_id|loan_amount|funded_amount|loan_term_years|interest_rate|monthly_installment|issue_date|       loan_status|      loan_purpose|          loan_title|         ingest_date|
+--------+--------------------+-----------+-------------+---------------+-------------+-------------------+----------+------------------+------------------+--------------------+--------------------+
|69642551|2209556c19f75a204...|    10000.0|      10000.0|              3|        10.78|             326.35|  Jan-2016|       Charged Off|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|69336966|9d0d53b8de898b446...|    12000.0|      12000.0|              3|        11.99|             398.52|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69226186|1a978870f4a9b4b43...|    28000.0|      28000.0|              3|         7.49|             870.85|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69084024|c66ccc99ff4372809...|    16500.0|      16500.0|              3|          9.8|             530.87|  Jan-2016|        Fully Paid|debt_consolidation|                null|2025-06-17 03:40:...|
|69422635|91d00a422248120a4...|    21000.0|      21000.0|              5|        10.78|              454.3|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69542436|b7539a5579e723b21...|    21000.0|      21000.0|              5|        14.85|             497.94|  Feb-2016|       Charged Off|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69492501|2fca945a60ef9246c...|    29000.0|      29000.0|              3|        11.48|             956.03|  Jan-2016|           Current|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|69522527|8a6dababd11fd592f...|    33000.0|      33000.0|              3|        13.44|            1118.91|  Jan-2016|        Fully Paid|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|68781653|3af72ce29e878edd7...|    18000.0|      18000.0|              5|        13.44|             413.63|  Jan-2016|        Fully Paid|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|69371334|4395baca2ea2d58a8...|    16000.0|      16000.0|              5|         7.91|             323.74|  Jan-2016|        Fully Paid|  home_improvement|    Home improvement|2025-06-17 03:40:...|
|68714098|45ac50a4b31e7c191...|    15000.0|      15000.0|              3|         6.99|             463.09|  Jan-2016|        Fully Paid|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69622505|a6a1b2d34c1f8d409...|    35000.0|      35000.0|              5|        19.89|             925.15|  Jan-2016|Late (31-120 days)|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69084020|aa1795bf3b4e526f4...|    12000.0|      12000.0|              3|        12.88|             403.64|  Jan-2016|       Charged Off|debt_consolidation|                null|2025-06-17 03:40:...|
|69552651|8e0e745628d1fb06e...|    24000.0|      24000.0|              5|        13.99|             558.32|  Jan-2016|           Current|debt_consolidation|  Debt consolidation|2025-06-17 03:40:...|
|69084037|232386e4de5f4542b...|     7200.0|       7200.0|              3|        11.48|             237.36|  Jan-2016|        Fully Paid|    major_purchase|                null|2025-06-17 03:40:...|
|69612464|2d3472c91f8b2a006...|     2500.0|       2500.0|              3|         8.49|              78.91|  Jan-2016|       Charged Off|           medical|    Medical expenses|2025-06-17 03:40:...|
|69562380|9abeb83f93fd934ff...|    22250.0|      22250.0|              5|        19.89|             588.13|  Jan-2016|           Current|       credit_card|Credit card refin...|2025-06-17 03:40:...|
|69422542|cb18b4a8c0

#### Create a table

In [21]:
loans_term_modified_df.createOrReplaceTempView("loans")

#### lets see what all distinct values we have in loan_purpose

In [22]:
spark.sql("select distinct(loan_purpose) from loans")


+--------------------+
|        loan_purpose|
+--------------------+
|        guaranteed!"|
|and if they are a...|
|never had any tro...|
|<br/><br/>Lending...|
|Bank of America c...|
|              stocks|
|please feel free ...|
|I became his prim...|
|              brakes|
|on one of the bus...|
|because of the na...|
|progressive multi...|
|but not much info...|
|I have eliminated...|
|TutoringOne is a ...|
|so I have to pay ...|
|you may feel conf...|
|Hilal Khalil Homa...|
|000 in debt on cr...|
|and the monthly p...|
+--------------------+
only showing top 20 rows

#### How many people have taken the loan for specific purpose

In [23]:
spark.sql("select loan_purpose, count(*) as total from loans group by loan_purpose order by total desc")

+--------------------+-------+
|        loan_purpose|  total|
+--------------------+-------+
|  debt_consolidation|1277790|
|         credit_card| 516926|
|    home_improvement| 150440|
|               other| 139413|
|      major_purchase|  50429|
|             medical|  27481|
|      small_business|  24659|
|                 car|  24009|
|            vacation|  15525|
|              moving|  15402|
|               house|  14131|
|             wedding|   2351|
|    renewable_energy|   1445|
|         educational|    412|
|<br/><br/>Lending...|      1|
|please feel free ...|      1|
|progressive multi...|      1|
|              stocks|      1|
|but not much info...|      1|
|I have eliminated...|      1|
+--------------------+-------+
only showing top 20 rows

In [24]:
loan_purpose_lookup = ["debt_consolidation","credit_card","home_improvement","other","major_purchase",
"medical","small_business","car","vacatioon","moving","house","wedding","renewable_energy","educational"]

In [25]:
from pyspark.sql.functions import when
loans_purpose_modified = loans_term_modified_df.withColumn("loan_purpose",when(col("loan_purpose").isin(loan_purpose_lookup), col("loan_purpose")).otherwise("other"))

#### Create a table

In [27]:
loans_purpose_modified.createOrReplaceTempView("loans")

#### Writing in Dataframe

In [39]:
from pyspark.sql.functions import count
loans_purpose_modified.groupBy("loan_purpose").agg(count("*")).alias("total").orderBy(col("total").desc())

AnalysisException: cannot resolve '`total`' given input columns: [total.count(1), total.loan_purpose];
'Sort ['total DESC NULLS LAST], true
+- SubqueryAlias total
   +- Aggregate [loan_purpose#744], [loan_purpose#744, count(1) AS count(1)#874L]
      +- Project [loan_id#150, member_id#151, loan_amount#152, funded_amount#153, loan_term_years#531, interest_rate#155, monthly_installment#156, issue_date#157, loan_status#158, CASE WHEN loan_purpose#159 IN (debt_consolidation,credit_card,home_improvement,other,major_purchase,medical,small_business,car,vacatioon,moving,house,wedding,renewable_energy,educational) THEN loan_purpose#159 ELSE other END AS loan_purpose#744, loan_title#160, ingest_date#172]
         +- Project [loan_id#150, member_id#151, loan_amount#152, funded_amount#153, loan_term_months#518 AS loan_term_years#531, interest_rate#155, monthly_installment#156, issue_date#157, loan_status#158, loan_purpose#159, loan_title#160, ingest_date#172]
            +- Project [loan_id#150, m

In [33]:
loans_purpose_modified.write \
.format("csv") \
.option("header",True) \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/cleaned/loans_data_csv") \
.save()

In [32]:
loans_purpose_modified.write \
.format("parquet") \
.option("header",True) \
.mode("overwrite") \
.option("path","/user/itv018355/lendingclubproject/cleaned/loans_data_parquet") \
.save()